# Spatial Join with Python

This tutorial will show you how to do field mapping with python in a Jupyter Notebook, how to use the command line for spatial joins, and we will even use a little bit of Pandas to display our data!

The first thing that we will need to do is import ArcPy and set our work environment. 

Here is the Google Drive link to the data for this tutorial https://drive.google.com/open?id=1HDo3IiRkYrSPpftHiAiJaPD4xBmifwB7 

Please download this data and unzip it to a folder of your choice. This folder will be our work environment.

I unzipped the data to C:\ex07_data, change the arcpy.env.workspace to where ever you unzipped your data.

This tutorial loosely follows the Advanced Spatial Join tutorial.

In [38]:
import arcpy 

#this will be where all the files we create go
arcpy.env.workspace = "C:\ex07_data"

#setting our path to our env workspace so we do not have to retype this long variable
path = arcpy.env.workspace

#### Let's list all the files in our workspace to see what we are working with

We will use the os package to do this.

In [39]:
import os

#let's make a blank list to put our file names in.
files = []

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        #if we just appended the file to the files list, we would not get the full directory that it is in
        files.append(os.path.join(r, file))

for f in files:
    print(f)

C:\ex07_data\haunted_places.csv
C:\ex07_data\vacant_buildings.csv
C:\ex07_data\bus_routes\bus_routes.CPG
C:\ex07_data\bus_routes\bus_routes.dbf
C:\ex07_data\bus_routes\bus_routes.prj
C:\ex07_data\bus_routes\bus_routes.sbn
C:\ex07_data\bus_routes\bus_routes.sbx
C:\ex07_data\bus_routes\bus_routes.shp
C:\ex07_data\bus_routes\bus_routes.shp.xml
C:\ex07_data\bus_routes\bus_routes.shx
C:\ex07_data\bus_routes\metadata\metadata.html
C:\ex07_data\bus_routes\metadata\metadata.xml
C:\ex07_data\bus_routes\metadata\preview.jpg
C:\ex07_data\bus_routes\metadata\Thumbs.db
C:\ex07_data\bus_routes\metadata\TransitDatasets_Commons.pdf
C:\ex07_data\district_council\district_councils.dbf
C:\ex07_data\district_council\district_councils.prj
C:\ex07_data\district_council\district_councils.shp
C:\ex07_data\district_council\district_councils.shp.xml
C:\ex07_data\district_council\district_councils.shx
C:\ex07_data\info\arc.dir
C:\ex07_data\light_rail\light_rails.CPG
C:\ex07_data\light_rail\light_rails.dbf
C:\ex0

### Cleaning the data

Our haunted_places and our vacant_buildings files are CSV's instead of shapefiles. 

Before converting these into shapefiles we should first print out the data to see what we are working with.

Let's convert these into dataframes with Pandas and print them.

In [75]:
import pandas as pd

hp = 'C:\ex07_data\haunted_places.csv'

vb = 'C:\ex07_data\\vacant_buildings.csv'

#pd.read_csv will read the file and convert it into a dataframe
hpDF = pd.read_csv(hp)

vbDF = pd.read_csv(vb)

# .head() will print the first five lines of the file

#this is the haunted places file
hpDF.head()

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097


#### With the haunted buildings csv, we want to extract the longitude and latitude to make a point shapefile. 

To do this we can use ArcPy's make XY event layer

In [ ]:

#hp is reading the CSV defined above, 'hp' is saving the file to a temporary layer.
arcpy.MakeXYEventLayer_management(hp, 'longitude', 'latitude', 'hp')

#save the event layer in our workspace to use later
arcpy.FeatureClassToFeatureClass_conversion('hp', path, 'hp.shp')

#defining the varible so the data can be easily accessed later
hpS = 'hp.shp'

In [77]:
#this will list the first five lines of the vacant building file
vbDF.head()

,ADDRESS,VACANT AS OF,DWELLING TYPE,VACANT BUILDING CATEGORY,WARD,DISTRICT,CENSUS TRACT,MAP LOCATION
0,663 3RD ST E,04/26/2011 12:00:00 AM,Multi-family Residential,2,7.0,4.0,34400.0,"(44.9547963325615, -93.071254461297)"
1,989 3RD ST E,11/01/2011 12:00:00 AM,Single Family Residential,2,7.0,4.0,34500.0,"(44.9568544056193, -93.058822503618)"
2,1457 3RD ST E,10/01/2014 12:00:00 AM,Single Family Residential,2,7.0,1.0,34602.0,"(44.9569709429740, -93.038835961878)"
3,249 4TH ST E,10/13/2016 12:00:00 AM,Commercial,2,2.0,17.0,34201.0,"(44.9487456956542, -93.085933397554)"
4,767 4TH ST E,01/09/2009 12:00:00 AM,Duplex,2,7.0,4.0,34500.0,"(44.9576793823968, -93.067077030399)"


### In our vacant buildings file we do not have latitude and longitude fields. Our coordinate data is in our Map Location column. 

Let's add latitude and longitude fields and populate them with this data. This can be done in either Pandas or ArcPy. For this example we will use ArcPy.

In [ ]:
#let's move the tables into the geodatabase here with python.
#we have to do this, otherwise we cannot modify the csv in ArcPy

#first let's make a geodatabase here
newGDB = 'newGDB.gdb'

arcpy.CreateFileGDB_management(path, newGDB)

arcpy.TableToTable_conversion('C:\ex07_data\\vacant_buildings.csv', path + '\\' +newGDB, 'vacant_buildings')

vbGDB = 'C:\\ex07_data\\newGDB.gdb\\vacant_buildings'

#Adding latitude and longitude fields
arcpy.AddField_management(vbGDB, 'lat', 'float')

arcpy.AddField_management(vbGDB, 'long', 'float')


#this will split the Map_location field into two parts, and put them in the correct field of the CSV
arcpy.CalculateField_management(vbGDB, 'lat', "!MAP_LOCATION![1:-1].split(',')[0]")

arcpy.CalculateField_management(vbGDB, 'long', "!MAP_LOCATION![1:-1].split(',')[1]")


#Lets create the layer like we did another for the haunted buildings
arcpy.MakeXYEventLayer_management(vbGDB, 'long', 'lat', 'vb')

#save the event layer in our workspace to use later
arcpy.FeatureClassToFeatureClass_conversion('vb', path, 'vb.shp')

### Let's add another field to our dataset

We will use this field for our spatial join, we could use the original 'Dwelling_Type' ('Dwelling_T') field, but this will show us how to use code blocks, and another tool in ArcPy. 

First, we need to add another field. 

Next, we'll define a function that will do in the CalculateField tool. 

This function will apply to every line of our attribute table.

Here is the python sytanx for this code:
arcpy.CalculateField_management(in_table, field, expression, {expression_type}, {code_block})



In [47]:
arcpy.AddField_management('vb.shp','DwellCode','Short')

#for each dwelling type, we will add a number to the DwellCode field
codeblock = """
def addCode(dType):
    if dType == 'Commercial':
        return 1
    if dType == 'Duplex':
        return 2
    if dType == 'Mixed Use':
        return 3
    if dType == 'Multi-family Residential':
        return 4
    if dType == 'Single Family Residential':
        return 5    
    """

#addCode(!Dwelling_T!) is calling our codeblock.
arcpy.CalculateField_management('vb.shp', 'DwellCode', 'addCode(!DWELLING_T!)', "PYTHON3", codeblock)

<Result 'C:\\ex07_data\\vb.shp'>

### Now that our data is cleaned and we've added some fields lets get to the Spatial Joins!

#### Let's find out what dwelling type of vacant building is most frequent in each district. We will do this with using a contains spatial join and field mapping.

Here is the python syntax for a spatial join equation.

arcpy.SpatialJoin_analysis(target_features, join_features, out_feature_class, {join_operation}, {join_type}, {field_mapping}, {match_option}, {search_radius}, {distance_field_name})

In [48]:
dc = 'C:\ex07_data\district_council\district_councils.shp'
vb = 'vb.shp'


# Create a new fieldmappings and add the two input feature classes.

#These are the two tables that we will join together.

#creating a field mapping object
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(dc)
fieldmappings.addTable(vb)
 
#the field we want to join is 'DWELLCODE'
dwelling_code = fieldmappings.findFieldMapIndex("DwellCode")
fieldmap = fieldmappings.getFieldMap(dwelling_code)
 
# Get the output field's properties as a field object
field = fieldmap.outputField
 
# Rename the field and pass the updated field object back into the field map
field.name = "mode_district"
field.aliasName = "mode_district"
fieldmap.outputField = field
 
# Set the merge rule to mean and then replace the old fieldmap in the mappings object
# with the updated one

#we code put many different merge rules here, such as mean, first, minimum, etc.
fieldmap.mergeRule = "mode"
fieldmappings.replaceFieldMap(dwelling_code, fieldmap)

#here is the final code, we are finding what district councils contain the vacant building.
arcpy.SpatialJoin_analysis(dc, vb, 'out.shp','JOIN_ONE_TO_ONE','KEEP_ALL',fieldmappings,'contains')

<Result 'C:\\ex07_data\\out.shp'>

### How many of each dwelling type are in each district?

In our Advanced Tutorial we did this by extracting each dwelling type into a new feature class and then used a contains spatial join.

We could do a similiar thing in ArcPy but let's take advantage of another python package called Pandas.

In [57]:
#first let's export the table we created in our spatial join to a csv
arcpy.TableToTable_conversion('vb.shp', 'C:\ex07_data', 'vb.csv')

#then lets read the csv with Pandas
vbDBF = pd.read_csv('C:\ex07_data\\vb.csv')

#lets print out the top of the dataframe to see what it looks like
vbDBF.head()

,FID,ADDRESS,VACANT_AS_,DWELLING_T,VACANT_BUI,WARD,DISTRICT,CENSUS_TRA,MAP_LOCATI,lat,long,DwellCode
0,0,663 3RD ST E,04/26/2011 12:00:00 AM,Multi-family Residential,2,7,4,34400,"(44.9547963325615, -93.071254461297)",44.954800,-93.071297,4
1,1,989 3RD ST E,11/01/2011 12:00:00 AM,Single Family Residential,2,7,4,34500,"(44.9568544056193, -93.058822503618)",44.956902,-93.058800,5
2,2,1457 3RD ST E,10/01/2014 12:00:00 AM,Single Family Residential,2,7,1,34602,"(44.9569709429740, -93.038835961878)",44.957001,-93.038803,5
3,3,249 4TH ST E,10/13/2016 12:00:00 AM,Commercial,2,2,17,34201,"(44.9487456956542, -93.085933397554)",44.948700,-93.085899,1
4,4,767 4TH ST E,01/09/2009 12:00:00 AM,Duplex,2,7,4,34500,"(44.9576793823968, -93.067077030399)",44.957699,-93.067100,2


In [64]:
#instead of doing a spatial join contains like in our Advanced Tutorial, we will group by the Dwelling Type and District. 

#Since we want to find the number of each Dwelling Type in each District we'll group by both. 
#Then we'll find the count of the Dwell Code field.
g = vbDBF.groupby(['DWELLING_T','DISTRICT']).DwellCode.count()

g.to_csv('dt_inDistricts.csv')  #dwelling type within district

g

DWELLING_T                 DISTRICT
Commercial                 2            3
                           3            7
                           4           11
                           5           11
                           6            3
                           7            9
                           8            4
                           9            4
                           10           1
                           11           9
                           12           1
                           13           3
                           17           4
Duplex                     1            2
                           2            4
                           3            2
                           4           14
                           5           25
                           6            9
                           7           14
                           8            6
                           9            2
                           10           

In [65]:
#Let's list what is in the file again so we can see the data we created 

files = []

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if  '.csv' in file:
        #if we just appended the file to the files list, we would not get the full directory that it is in
            files.append(os.path.join(r, file))

for f in files:
    print(f)
    
#the files saved as .dbf so we will search for those.

#we can see the files outputted here that we want

C:\ex07_data\haunted_places.csv
C:\ex07_data\vacant_buildings.csv
C:\ex07_data\vb.csv
C:\ex07_data\vb.csv.xml
C:\ex07_data\vs.csv
C:\ex07_data\vs.csv.xml


## Spatial Joins in the Command Line

We can do anything that we have done above in the command line, except for print out the name of the files, and printing out the head of our data frames. 

Lets try it!

Lets start with a one-liner. To do this, open you the 'Python Command Prompt" from the ArcPro folder in your start menu.

This is what the terminal should look like when we open it.
![Image](https://ajmcgraw.github.com/ajmcgraw.github.io/images_website/arcPro_Env.PNG)

Type 'python' to activate python.

###### Let's set our environment and then get to coding, copy and paste the following lines,

<python> import arcpy 

arcpy.env.workspace = "C:\ex07_data" <python>
    
###### Then let's do a simple Spatial Join, let's see what haunted places are within the district councils of St. Paul

arcpy.SpatialJoin_analysis('hp.shp','C:\ex07_data\district_council\district_councils.shp','hp_withinSTP.shp','JOIN_ONE_TO_ONE', 'KEEP_COMMON', 'WITHIN')

![Image](https://ajmcgraw.github.com/ajmcgraw.github.io/images_website/arcPro_script.PNG)

##### We can also run the script we ran above in the terminal. Let's make it into a .py script to run it easier.

Run the cell below to create a .py script, remember to change the environment work space before doing this.

In [89]:
pyText = """ import arcpy 

arcpy.env.workspace = "C:\ex07_data"

dc = 'C:\ex07_data\district_council\district_councils.shp'
vb = 'vb.shp'


# Create a new fieldmappings and add the two input feature classes.

#These are the two tables that we will join together.

#creating a field mapping object
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(dc)
fieldmappings.addTable(vb)
 
#the field we want to join is 'DWELLCODE'
dwelling_code = fieldmappings.findFieldMapIndex("DwellCode")
fieldmap = fieldmappings.getFieldMap(dwelling_code)
 
# Get the output field's properties as a field object
field = fieldmap.outputField
 
# Rename the field and pass the updated field object back into the field map
field.name = "mode_district"
field.aliasName = "mode_district"
fieldmap.outputField = field
 
# Set the merge rule to mean and then replace the old fieldmap in the mappings object
# with the updated one

#we code put many different merge rules here, such as mean, first, minimum, etc.
fieldmap.mergeRule = "mode"
fieldmappings.replaceFieldMap(dwelling_code, fieldmap)

#here is the final code, we are finding what district councils contain the vacant building.
arcpy.SpatialJoin_analysis(dc, vb, 'output.shp','JOIN_ONE_TO_ONE','KEEP_ALL',fieldmappings,'contains')"""

f = open('C:\ex07_data\script.py', 'w')

f.write(pyText)

f.close()

### Now let's run the script we created on the command line

![Image](https://ajmcgraw.github.com/ajmcgraw.github.io/images_website/arcPro_oncmdline.PNG)




#### After this tutorial you should have a good idea about how to do spatial joins in python. 

Here are some additional resources

https://pro.arcgis.com/en/pro-app/tool-reference/analysis/spatial-join.htm

https://pro.arcgis.com/en/pro-app/arcpy/main/arcgis-pro-arcpy-reference.htm

https://pro.arcgis.com/en/pro-app/arcpy/get-started/mapping-fields.htm